In [1]:
!pip install torch transformers datasets scikit-learn pandas numpy matplotlib
!pip install nltk


In [2]:
!pip install --upgrade transformers


In [3]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset


In [4]:
df = pd.read_csv("mental_health_dataset_1000.csv")
df.head()


,text,depression,anxiety,stress
0,I hate myself,0,0,1
1,I'm burnt out,0,0,1
2,I had lunch today,0,0,0
3,I'm tired of everything,0,0,1
4,Nothing makes me happy,1,0,1


In [5]:
df['labels'] = df[['depression','anxiety','stress']].values.tolist()
df = df[['text','labels']]


In [6]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


In [8]:
class MentalHealthDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=128):
        self.tokenizer = tokenizer
        self.texts = dataframe['text'].tolist()
        self.labels = dataframe['labels'].tolist()
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(text,
                                  truncation=True,
                                  padding="max_length",
                                  max_length=self.max_len,
                                  return_tensors="pt")

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(self.labels[idx], dtype=torch.float)
        }


In [9]:
train_dataset = MentalHealthDataset(train_df, tokenizer)
test_dataset  = MentalHealthDataset(test_df, tokenizer)


In [10]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=3,  
    problem_type="multi_label_classification"
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
training_args = TrainingArguments(
    output_dir="./bert_model",
    eval_strategy="epoch",        # <-- use eval_strategy
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
)


In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()


C:\Users\NAKSHATRA\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,No log,0.367055
2,No log,0.320027
3,No log,0.316672
4,No log,0.315101
5,0.372000,0.314791


C:\Users\NAKSHATRA\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\NAKSHATRA\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\NAKSHATRA\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\NAKSHATRA\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=500, training_loss=0.37196951293945313, metrics={'train_runtime': 31952.753, 'train_samples_per_second': 0.125, 'train_steps_per_second': 0.016, 'total_flos': 263113417728000.0, 'train_loss': 0.37196951293945313, 'epoch': 5.0})

In [18]:
from sklearn.metrics import classification_report, accuracy_score

# Predictions
preds = trainer.predict(test_dataset)
y_pred = torch.sigmoid(torch.tensor(preds.predictions)).numpy()
y_pred = (y_pred > 0.5).astype(int)

# True labels
y_true = np.array(test_df['labels'].tolist())

# Print classification report
print(classification_report(
    y_true, 
    y_pred,
    target_names=['depression','anxiety','stress']
))

# ---- ADD ACCURACY METRICS HERE ----

# Exact match accuracy (multi-label accuracy)
exact_acc = accuracy_score(y_true, y_pred)

# Micro accuracy (label-wise accuracy)
micro_acc = (y_true == y_pred).mean()

print("Exact Match Accuracy:", round(exact_acc, 4))
print("Micro Accuracy:", round(micro_acc, 4))


C:\Users\NAKSHATRA\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


              precision    recall  f1-score   support

  depression       0.94      0.90      0.92        67
     anxiety       0.89      0.76      0.82        55
      stress       0.95      0.62      0.75       102

   micro avg       0.93      0.74      0.82       224
   macro avg       0.93      0.76      0.83       224
weighted avg       0.93      0.74      0.82       224
 samples avg       0.82      0.72      0.76       224

Exact Match Accuracy: 0.74
Micro Accuracy: 0.8817


C:\Users\NAKSHATRA\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\NAKSHATRA\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
model.save_pretrained("saved_model")
tokenizer.save_pretrained("saved_model")


('saved_model\\tokenizer_config.json',
 'saved_model\\special_tokens_map.json',
 'saved_model\\vocab.txt',
 'saved_model\\added_tokens.json')

In [20]:
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    probs = torch.sigmoid(outputs.logits).detach().numpy()[0]
    
    return {
        "depression": float(probs[0]),
        "anxiety": float(probs[1]),
        "stress": float(probs[2])
    }

predict("I feel hopeless and tired of life")


{'depression': 0.7081438899040222,
 'anxiety': 0.021500475704669952,
 'stress': 0.7619432210922241}